In [80]:
using JuMP
import LinearAlgebra
import SCS
import Random

In [81]:
N, d = 5, 2

(5, 2)

In [82]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

ρ = [random_state(d) for i in 1:N]

5-element Vector{LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}}:
 [0.1179273425604257 + 0.0im 0.24478912679331744 - 0.18036031823375892im; 0.24478912679331744 + 0.18036031823375892im 0.8820726574395743 + 0.0im]
 [0.7519808998610745 + 0.0im 0.16013356626193226 + 0.03036016590370519im; 0.16013356626193226 - 0.03036016590370519im 0.2480191001389255 + 0.0im]
 [0.33212431410756055 + 0.0im -0.30450104936885763 - 0.25777720018644174im; -0.30450104936885763 + 0.25777720018644174im 0.6678756858924395 + 0.0im]
 [0.6179389246516989 + 0.0im -0.4260278853437451 + 0.04594319908498713im; -0.4260278853437451 - 0.04594319908498713im 0.38206107534830097 + 0.0im]
 [0.7773832653285794 + 0.0im 0.26039299936342064 + 0.13647158405580734im; 0.26039299936342064 - 0.13647158405580734im 0.22261673467142062 + 0.0im]

In [83]:
Random.seed!(7);
q = rand(N)
sumq = sum(q)
q = q / sumq

# q = [0.5,0.5]

5-element Vector{Float64}:
 0.0766016696574427
 0.22597061682377695
 0.2547616658912036
 0.14635568463058968
 0.296310362996987

In [84]:
model = Model(SCS.Optimizer)
set_silent(model)

In [85]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

5-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]
 [_[9] _[10] + _[12] im; _[10] - _[12] im _[11]]
 [_[13] _[14] + _[16] im; _[14] - _[16] im _[15]]
 [_[17] _[18] + _[20] im; _[18] - _[20] im _[19]]

In [86]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] + _[9] + _[13] + _[17] - 1                                                    …  _[2] + _[6] + _[10] + _[14] + _[18] + _[4] im + _[8] im + _[12] im + _[16] im + _[20] im
 _[2] + _[6] + _[10] + _[14] + _[18] - _[4] im - _[8] im - _[12] im - _[16] im - _[20] im     _[3] + _[7] + _[11] + _[15] + _[19] - 1] ∈ Zeros()

In [87]:
@objective(
    model,
    Max,
    sum(q[i] * real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.0018066862676787622 _[1] + 0.007500502330542223 _[2] - 0.005526360606661456 _[4] + 0.013513647663809778 _[3] + 0.03398511755626117 _[5] + 0.014474192296959995 _[6] + 0.0027442021664529853 _[8] + 0.01120900580849422 _[7] + 0.0169225087090031 _[9] - 0.03103007784113192 _[10] - 0.02626869957930727 _[12] + 0.03402982446923762 _[11] + 0.01808777487545795 _[13] - 0.02494064113248247 _[14] + 0.0026896193424811093 _[16] + 0.01118336205065998 _[15] + 0.04606934350745888 _[17] + 0.030862857665299746 _[18] + 0.016175177844140038 _[20] + 0.013192729091938516 _[19]

In [88]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.00205e-01
  Dual objective value : 1.00192e-01

* Work counters
  Solve time (sec)   : 4.87401e-03


In [89]:
objective_value(model)

0.10020497623830785

In [90]:
value.(E[1])

2×2 Matrix{ComplexF64}:
 -1.17398e-5+0.0im         4.26582e-5+2.9219e-5im
  4.26582e-5-2.9219e-5im  -8.10565e-5+0.0im

In [91]:
conj(value.(E[1]))

2×2 Matrix{ComplexF64}:
 -1.17398e-5-0.0im         4.26582e-5-2.9219e-5im
  4.26582e-5+2.9219e-5im  -8.10565e-5-0.0im

In [ ]:
# #Optimality condition
for i in  1:N
    for j in (i+1):N
        print(value.(E[i]) * (q[i] * ρ[i] - q[j] * ρ[j]) * value.(E[j]))
        print("\n")
    end
end

ComplexF64[3.99362540883624e-11 - 1.7179484827559902e-13im -4.422435547938972e-11 - 1.529884895323742e-11im; -4.139047342975439e-11 + 1.3636920873554035e-11im -1.3552051173326323e-10 + 1.7179475525693467e-13im]
ComplexF64[4.191508476493772e-6 + 7.586507946769988e-8im -6.375417799508489e-6 - 4.543790422687032e-6im; -7.103595317682587e-6 + 4.918307161939963e-6im 1.6135187228831923e-5 - 7.231571058318425e-8im]
ComplexF64[-4.832056774275865e-11 - 6.988871498018648e-13im 1.0594176399730141e-10 + 2.0339382535097233e-10im; 1.0800690503904092e-10 - 2.036727101303167e-10im -6.070536477038405e-10 + 6.988867123290642e-13im]
ComplexF64[-7.291917282254567e-7 + 5.295274850179085e-7im -4.825500756270673e-7 + 1.2922397161919505e-8im; -5.244397282275116e-6 + 2.3892615158891114e-6im -3.0393592030074518e-6 - 5.330775296079642e-7im]
ComplexF64[4.4585898204665516e-6 + 2.5072102866164974e-8im -6.840433563391231e-6 - 4.747677330510693e-6im; -6.7555388251735465e-6 + 4.649213630245677e-6im 1.5315130534130524e-

In [93]:
solution = [value.(e) for e in E]

5-element Vector{Matrix{ComplexF64}}:
 [-1.1739840657904685e-5 + 0.0im 4.2658184583799855e-5 + 2.921896058330328e-5im; 4.2658184583799855e-5 - 2.921896058330328e-5im -8.105654104442194e-5 + 0.0im]
 [-1.2236470070209153e-5 + 0.0im 4.260436999762662e-5 + 2.915513828617942e-5im; 4.260436999762662e-5 - 2.915513828617942e-5im -8.102097209663891e-5 + 0.0im]
 [0.22294402274831474 + 0.0im -0.34334572865146734 - 0.23545358303980066im; -0.34334572865146734 + 0.23545358303980066im 0.7775071067128232 + 0.0im]
 [-1.1991116656683438e-5 + 0.0im 4.290852647129595e-5 + 2.9155559491028954e-5im; 4.290852647129595e-5 - 2.9155559491028954e-5im -8.102057632088214e-5 + 0.0im]
 [0.7770919949757022 + 0.0im 0.3432172981428008 + 0.23536587618483232im; 0.3432172981428008 - 0.23536587618483232im 0.22273633857597464 + 0.0im]